In [ ]:
from sentence_transformers import SentenceTransformer
from underthesea import word_tokenize
import numpy as np
import unicodedata
import json
import csv
import re
from LLM import Process_LLM

def normalize_text(text):
    text = text.replace('đ', 'd').replace('Đ', 'D')
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    text = text.lower().strip()
    return text
def extract_keywords(text):
    def is_number(text):
        return bool(re.fullmatch(r'[\d,. ]+', text))
    specialchars = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '..', '...', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
    specialwords = ["giấy tờ", "thủ tục", "giấy", "gì", "cần", "nào", "sắp", "đang", "sẽ", "của", "bị", "hoặc", "với", "và", "thì", "muốn", "gì", "mình", "tôi", "phải", "làm sao", "để", "cho", "làm", "như", "đối với", "từ", "theo", "là", "được", "ở", "đã", "về", "có", "các", "tại", "đến", "vào", "do", "vì", "bởi vì", "thuộc"]
    words = word_tokenize(text)
    words = [w.lower().strip() for w in words]
    words = list(set(words))
    words = [w for w in words if not is_number(w)]
    words = [w for w in words if w not in specialchars]
    words = [w for w in words if w not in specialwords]
    words_normalized = [normalize_text(w) for w in words if " " in w]
    return list(set(words + words_normalized))

In [ ]:
# ---------- Load models and embeddings ----------
model_e5 = SentenceTransformer("onelevelstudio/M-E5-BASE")
model_mpnet = SentenceTransformer("onelevelstudio/M-MPNET-BASE")
embs_e5 = np.load("url/embs_e5")
embs_mpnet = np.load("url/embs_mpnet")

# ---------- Load thutucs from cache ----------
with open('url/cache', mode='r', newline='', encoding='utf-8') as f:
    thutucs = list(csv.DictReader(f))
    for i in range(len(thutucs)):
        thutucs[i]["keywords"] = extract_keywords(thutucs[i]["Tên thủ tục"])

In [ ]:
test_questions = [
"mình sắp khởi nghiệp cần giấy tờ gì?",
"tôi muốn mua đất thì cần làm gì?",
"tôi sắp cưới vợ thì phải làm như nào?",
"tôi sắp lập gia đình thì cần làm gì?",
"vợ tôi sắp sinh con thủ tục nào?",
"thủ tục xây nhà cấp 3 cấp 4?",
"làm sao để phúc khảo bài thi thpt?",
"làm sao để tố cáo hành vi vi phạm pháp luật?",
"tôi muốn ly dị chồng, tôi phải làm gì?",
"tôi muốn ly hôn chồng, tôi phải làm gì?",
"tôi muốn chuyển hộ khẩu cho con của tôi",
"làm sao để đăng ký nhập học cho con của tôi?",
"dang ky ket hon",
"đăng ký kết hôn",
"đăng ký",
"tố cáo xã",
]

In [ ]:
def retrieve_idx_semantic(text, pre_embs, emb_model, top=5):
    q_emb = emb_model.encode(text)
    similarities = emb_model.similarity(q_emb, pre_embs)[0]
    top_5_idx = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top]
    return top_5_idx

def retrieve_idx_exactmatch(text, thutucs, top=5):
    thutuc_scores = []
    for e in thutucs:
        score = 0
        # ----------
        if normalize_text(text) in normalize_text(e["Tên thủ tục"]):
            score += 1
        if text.lower().strip() in e["Tên thủ tục"].lower():
            score += 1
        # ----------
        thutuc_scores.append(score)
    thutuc_scores_idx = sorted(range(len(thutuc_scores)), key=lambda i: thutuc_scores[i], reverse=True)
    scores = [(idx, thutuc_scores[idx]) for idx in thutuc_scores_idx]
    scores = [e for e in scores if e[1] != 0]
    return [e[0] for e in scores][:min(len(scores), top)]

def retrieve_idx_keywordmatch(text, thutucs, top=5):
    q_keywords = extract_keywords(text)
    # print(q_keywords)
    thutuc_scores = []
    for e in thutucs:
        score = 0
        # ----------
        for k in q_keywords:
            if k in e["keywords"]:
                score += 1
        # ----------
        thutuc_scores.append(score)
    thutuc_scores_idx = sorted(range(len(thutuc_scores)), key=lambda i: thutuc_scores[i], reverse=True)
    scores = [(idx, thutuc_scores[idx]) for idx in thutuc_scores_idx]
    scores = [e for e in scores if e[1] != 0]
    return [e[0] for e in scores][:min(len(scores), top)]



for input_text in test_questions:

    print("="*100)
    print(f"> {input_text}")

    # ----------------------------------------------------------------------------------------------------
    res_exactmatch_idx = retrieve_idx_exactmatch(text=input_text, thutucs=thutucs, top=3)
    res_keywordmatch_idx = retrieve_idx_keywordmatch(text=input_text, thutucs=thutucs, top=3)
    res_semantic_idx_1 = retrieve_idx_semantic(text=input_text, pre_embs=embs_e5, emb_model=model_e5, top=3)
    res_semantic_idx_2 = retrieve_idx_semantic(text=input_text, pre_embs=embs_mpnet, emb_model=model_mpnet, top=3)

    # ----------------------------------------------------------------------------------------------------
    # Case 1: There is exact match -> only keep the exact match
    if len(res_exactmatch_idx) > 0:
        res_all_idx = res_exactmatch_idx
    # Case 2: There is no exact match -> keyword + sementic
    else:
        res_all_idx = res_keywordmatch_idx + res_semantic_idx_1 + res_semantic_idx_2

    # ----------------------------------------------------------------------------------------------------
    p_danhsachthutuc = [{"Mã chuẩn": thutucs[idx]["Mã chuẩn"], "Tên thủ tục": thutucs[idx]["Tên thủ tục"]} for idx in res_all_idx]
    p_json_schema = """\
    {
        "type": "object",
        "properties": {
            "Mã chuẩn": {"type": "string", "description": "Mã chuẩn của thủ tục liên quan nhất"},
            "Tên thủ tục": {"type": "string", "description": "Tên của thủ tục liên quan nhất"}
        }
    }"""
    prompt_1 = f"""\
    Bạn sẽ được cung cấp: (1) Câu hỏi của người dùng, (2) Danh sách thủ tục hiện có, và (3) Schema cấu trúc của kết quả.
    Nhiệm vụ của bạn là: (4) Trích xuất duy nhất 1 thủ tục liên quan nhất đến câu hỏi của người dùng.

    ### (1) Câu hỏi của người dùng:
    "{input_text}"

    ### (2) Danh sách thủ tục hiện có:
    {p_danhsachthutuc}

    ### (3) Schema cấu trúc của kết quả:
    {p_json_schema}

    ### (4) Nhiệm vụ:
    Từ câu hỏi của người dùng, tìm ra duy nhất 1 thủ tục liên quan nhất đến câu hỏi của người dùng, tuân thủ schema một cách chính xác.
    Định dạng kết quả: Không giải thích, không bình luận, không văn bản thừa. Chỉ trả về kết quả JSON hợp lệ. Bắt đầu bằng "{{", kết thúc bằng "}}".
    """
    # print(prompt_1)

    # ----------------------------------------------------------------------------------------------------

    for i in range(5):
        llm_text_1 = Process_LLM(prompt=prompt_1, vendor="ollama")
        regex_match = re.search(r'\{.*\}', llm_text_1, re.S)
        if regex_match:
            try:
                llm_object_1 = json.loads(regex_match.group())
                # --------------------------------------------------
                idx = next((i for i, d in enumerate(thutucs) if d["Mã chuẩn"] == llm_object_1["Mã chuẩn"].strip()), -1)
                if idx != -1:
                    print(f"✅ {thutucs[idx]['Tên thủ tục']} ({thutucs[idx]['thutuc_Link']})")
                    # --------------------------------------------------
                    # eee = thutucs[idx]
                    # thutuc_content = f"""\
                    # # Thủ tục: {eee['Tên thủ tục']}
                    # \n### Trình tự thực hiện:
                    # {eee['thutuc_Trình tự thực hiện']}
                    # \n### Cách thức thực hiện:
                    # {eee['thutuc_Cách thức thực hiện']}
                    # \n### Thành phần hồ sơ:
                    # {eee['thutuc_Thành phần hồ sơ']}
                    # \n### Thời gian giải quyết:
                    # {eee['thutuc_Thời gian giải quyết']}
                    # \n### Đối tượng thực hiện:
                    # {eee['thutuc_Đối tượng thực hiện']}
                    # \n### Cơ quan thực hiện:
                    # {eee['thutuc_Cơ quan thực hiện']}
                    # \n### Kết quả:
                    # {eee['thutuc_Kết quả']}
                    # \n### Phí, lệ phí:
                    # {eee['thutuc_Phí, lệ phí']}
                    # \n### Tên mẫu đơn, tờ khai:
                    # {eee['thutuc_Tên mẫu đơn, tờ khai']}
                    # \n### Yêu cầu, điều kiện:
                    # {eee['thutuc_Yêu cầu, điều kiện']}
                    # \n### Căn cứ pháp lý:
                    # {eee['thutuc_Căn cứ pháp lý']}
                    # """
                    # prompt_2 = f"""\
                    # Bạn sẽ được cung cấp: (1) Câu hỏi của người dùng, và (2) Nội dung thủ tục.
                    # Nhiệm vụ của bạn là: (3) Trả lời câu hỏi của người dùng dựa trên nội dung thủ tục một cách ngắn gọn và chính xác, chỉ duy nhất dựa trên thủ tục được cung cấp, chỉ sử dụng ngôn ngữ tiếng Việt.
                    # Nếu không có thông tin cần có trong thủ tục để trả lời câu hỏi, chỉ cần trả về "Mình chưa trả lời được câu hỏi này".

                    # (1) Câu hỏi của người dùng:
                    # "{input_text}"

                    # (2) Nội dung thủ tục:
                    # \"\"\"
                    # {thutuc_content}
                    # \"\"\"

                    # (3) Nhiệm vụ:
                    # Dựa trên nội dung thủ tục, hãy trả lời câu hỏi của người dùng "{input_text}" một cách ngắn gọn và chính xác, chỉ sử dụng ngôn ngữ tiếng Việt.
                    # """
                    # # print(prompt_2)
                    # llm_text_2 = Process_LLM(prompt=prompt_2, vendor="ollama")
                    # print(llm_text_2)
                    # --------------------------------------------------
                    break
                # --------------------------------------------------
            except:
                pass